In [47]:
import pandas as pd
import requests
import json
import csv
from src.data.utilities import weather_mapping
import os


class DataPoint:
    def __init__(self, api_key):
        self.api_key = api_key
        self.base_url = 'http://datapoint.metoffice.gov.uk/public/data/val/wxfcs/all/json/'

    def get_location_id(self, location_name):
        try:
            response = requests.get(f'{self.base_url}sitelist?key={self.api_key}')
            if response.status_code == 200:
                data = response.json()
    
                for location in data['Locations']['Location']:
                    if location['name'] == location_name:
                        return location['id']
    
                raise ValueError(f'Location "{location_name}" not found.')
            else:
                print(f'error: status code is {response.status_code}')
        except Exception as e:
            print(f'Error: {e}')

    def get_forecast_data(self, location_ids):
        forecast_data = pd.DataFrame()

        for location_id in location_ids:
            try:
                response = requests.get(f'{self.base_url}{location_id}?res=3hourly&key={self.api_key}')
                if response.status_code == 200:
                    data = response.json()
    
                    location_name = data['SiteRep']['DV']['Location']['name']
                    for day_data in data['SiteRep']['DV']['Location']['Period']:
                        forecast = day_data['Rep']

                        forecast_df = pd.DataFrame(forecast)
                        forecast_df.insert(0, 'Location', location_name)
                        forecast_df.insert(1, 'Day', day_data['value'])
                        forecast_data = pd.concat([forecast_data, forecast_df], ignore_index=True)
                else:
                    print(f'error: status code is {response.status_code}')
            except Exception as e:
                print(f'Error fetching data for location {location_id}: {e}')

        return forecast_data
    
    def save_to_csv(self, data, file_name, folder_path):
        try:
            if not os.path.exists(folder_path):
                os.makedirs(folder_path)

            file_path = os.path.join(folder_path, file_name)
            data.to_csv(file_path, index=False)
            print(f"Data saved successfully to {file_path}")
        except Exception as e:
            print(f"Error saving data to CSV: {e}")
            
    def fix_columns(self, df):
        df.rename(columns={'D': 'Wind Direction', 'F': 'Feels Like Temperature', 'G': 'Wind Gust', 'H': 'Screen Relative Humidity', 'Pp': 'Precipitation Probability', 'S': 'Wind Speed', 'T': 'Temperature', 'V': 'Visibility', 'W': 'Weather Type', 'U': 'Max UV Index'}, inplace=True)
        del df['$']
        df['Weather Type'] = df['Weather Type'].map(weather_mapping)
        return df


# Example usage:
api_key = "2155229d-fef4-4b2b-ac46-5d83b5fa9272"
data = DataPoint(api_key)

# Example of getting location ID for "Liverpool"
location_id = data.get_location_id("Hinton Ampner Garden")
print(f"Location ID for Hinton Ampner Garden: {location_id}")

# Example of getting forecast data for multiple locations (only 30 for simplicity)
location_ids = ['310012', '310009', '310004', '310042', '310046', '310013', '351905',                    '310069', '310016', '14', '26', '33', '3006', '3068', '3075', '3081', '3002',                '3005', '310024', '310022', '310025', '310047', '3908', '310035', '310026',                  '310048', '310031', '310037', '310011', '310006']

forecast_data = data.get_forecast_data(location_ids)

data.save_to_csv(forecast_data, file_name='Raw Data.csv', folder_path='/Users/seanwhite/OneDrive - University of Greenwich/Documents/Advanced Programming/Coursework/CourseworkReport&Code/data/raw/')

processed_data = data.fix_columns(forecast_data)
data.save_to_csv(processed_data, file_name='Processed Data.csv', folder_path='/Users/seanwhite/OneDrive - University of Greenwich/Documents/Advanced Programming/Coursework/CourseworkReport&Code/data/processed/')
pd.DataFrame(processed_data)

Location ID for Hinton Ampner Garden: 351905
Data saved successfully to /Users/seanwhite/OneDrive - University of Greenwich/Documents/Advanced Programming/Coursework/CourseworkReport&Code/data/raw/Raw Data.csv
Data saved successfully to /Users/seanwhite/OneDrive - University of Greenwich/Documents/Advanced Programming/Coursework/CourseworkReport&Code/data/processed/Processed Data.csv


,Location,Day,Wind Direction,Feels Like Temperature,Wind Gust,Screen Relative Humidity,Precipitation Probability,Wind Speed,Temperature,Visibility,Weather Type,Max UV Index
0,LIVERPOOL,2024-03-07Z,E,5,27,72,7,16,8,MO,Cloudy,0
1,LIVERPOOL,2024-03-07Z,E,3,22,74,15,11,7,GO,Cloudy,0
2,LIVERPOOL,2024-03-08Z,ESE,1,22,79,4,13,5,GO,Cloudy,0
3,LIVERPOOL,2024-03-08Z,E,2,22,81,8,13,6,GO,Cloudy,0
4,LIVERPOOL,2024-03-08Z,E,2,25,79,8,13,6,GO,Cloudy,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1015,COVENTRY,2024-03-11Z,NE,3,16,93,15,7,5,MO,Cloudy,1
1016,COVENTRY,2024-03-11Z,NE,6,16,83,12,7,8,GO,Cloudy,2
1017,COVENTRY,2024-03-11Z,NE,8,13,74,12,7,9,VG,Cloudy,1
1018,COVENTRY,2024-03-11Z,NNE,6,11,83,7,7,7,VG,Cloudy,0


In [2]:
list = []
for i in range(0, 34, +1):
    list.append(i)
print(list)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33]
